In [2]:
import pandas as pd
import requests
import random
import json
import time
import glob
import os

In [3]:
tool2tag = {
    'Azure Machine Learning': {'azure-machine-learning-service', 'azure-machine-learning-studio', 'azure-machine-learning-workbench'},
    'Vertex AI': {'google-cloud-vertex-ai', 'vertex-ai-pipeline'},
    'Amazon SageMaker': {'amazon-sagemaker', 'amazon-sagemaker-experiments'},
    'MLFlow': {'mlflow'},
    'DVC': {'dvc'},
    'Weights & Biases': {'wandb'},
    'ClearML': {'clearml'},
    'Comet': {'comet-ml'},
    'Neptune': {'neptune'},
    'Pachyderm': {'pachyderm'}
}

In [4]:
path_dataset = '../Dataset'

path_so = os.path.join(path_dataset, 'Stack Overflow')
path_to = os.path.join(path_dataset, 'Tool-specific Others')

path_so_raw = os.path.join(path_so, 'Raw')
path_to_raw = os.path.join(path_to, 'Raw')
path_so_filtered = os.path.join(path_so, 'Filtered')
path_to_filtered = os.path.join(path_to, 'Filtered')
path_so_sampled = os.path.join(path_so, 'Sampled')
path_to_sampled = os.path.join(path_to, 'Sampled')
path_so_sampled_questions = os.path.join(path_so_sampled, 'Questions')
path_to_sampled_questions = os.path.join(path_to_sampled, 'Questions')
path_so_sampled_answered = os.path.join(path_so_sampled, 'Answered')
path_to_sampled_answered = os.path.join(path_to_sampled, 'Answered')

if not os.path.exists(path_dataset):
    os.makedirs(path_dataset)

if not os.path.isdir(path_so):
    os.mkdir(path_so)

if not os.path.isdir(path_to):
    os.mkdir(path_to)

if not os.path.isdir(path_so_raw):
    os.mkdir(path_so_raw)

if not os.path.isdir(path_to_raw):
    os.mkdir(path_to_raw)

if not os.path.isdir(path_so_filtered):
    os.mkdir(path_so_filtered)

if not os.path.isdir(path_to_filtered):
    os.mkdir(path_to_filtered)

if not os.path.isdir(path_so_sampled):
    os.mkdir(path_so_sampled)

if not os.path.isdir(path_to_sampled):
    os.mkdir(path_to_sampled)

if not os.path.isdir(path_so_sampled_questions):
    os.mkdir(path_so_sampled_questions)

if not os.path.isdir(path_to_sampled_questions):
    os.mkdir(path_to_sampled_questions)

if not os.path.isdir(path_so_sampled_answered):
    os.mkdir(path_so_sampled_answered)

if not os.path.isdir(path_to_sampled_answered):
    os.mkdir(path_to_sampled_answered)

In [8]:
page = -1
base_url = 'https://my.guild.ai/'
page_suffix = 'c/troubleshooting/6.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_time'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_raw, 'Guild AI.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [7]:
page = -1
base_url = 'https://community.wandb.ai/'
page_suffix = 'c/w-b-support/36.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_time'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_raw, 'Weights & Biases.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [5]:
page = -1
base_url = 'https://community.sigopt.com/'
page_suffix = 'c/general-discussion/9.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_time'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_raw, 'SigOpt.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [5]:
page = -1
base_url = 'https://discuss.dvc.org/'
page_suffix = 'c/questions/9.json?page='
post_list = []

while True:
    page = page + 1
    page_url = base_url + page_suffix + str(page)
    topic_list = requests.get(page_url).json()['topic_list']

    for topic in topic_list['topics']:
        post_url = base_url + 't/' + \
            topic['slug'] + '/' + str(topic['id']) + '.json'

        post = {}
        post['Question_title'] = topic['title']
        post['Question_link'] = post_url
        post['Question_creation_time'] = topic['created_at']
        post['Question_answer_count'] = topic['posts_count'] - 1
        post['Question_upvote_count'] = topic['like_count']
        post['Question_view_count'] = topic['views']
        post['Question_has_accepted_answer'] = topic['has_accepted_answer']
        comments = requests.get(post_url).json()['post_stream']['posts']
        post['Question_body'] = comments[0]['cooked']
        post['Answers'] = comments[1:]
        post_list.append(post)

        time.sleep(2)

    if 'more_topics_url' not in topic_list.keys():
        break

json_post_list = json.dumps(post_list, indent='\t')
with open(os.path.join(path_to_raw, 'DVC.json'), 'w') as outfile:
    outfile.write(json_post_list)

In [28]:
df_to = pd.DataFrame()
# exclude tool-specific posts with negative upvote count
for file_name in glob.glob(os.path.join(path_to_raw, '*.json')):
    repos = pd.read_json(file_name)
    if 'Question_upvote_count' in repos.columns:
        repos = repos[repos['Question_upvote_count'] > -1]
    repos['Tool'] = os.path.split(file_name)[1].split('.')[0]
    df_to = pd.concat([df_to, repos], ignore_index=True)
df_to.to_json(os.path.join(path_to_filtered,
              'non_negative_scored.json'), orient='records', indent=4)


In [29]:
df_to_answered = df_to[df_to['Question_answer_count'] > 0]
for tool in df_to_answered['Tool'].unique().tolist():
    number_accepted_answer = df_to_answered[df_to_answered['Tool']
                                            == tool]['Question_has_accepted_answer'].sum()
    if number_accepted_answer > 0:
        df_to_answered = df_to_answered.drop(df_to_answered[(df_to_answered['Tool'] == tool) & (
            df_to_answered['Question_has_accepted_answer'] == False)].index)
df_to_answered.to_json(os.path.join(
    path_to_filtered, 'completed_non_negative_scored.json'), orient='records', indent=4)


In [94]:
# keep only posts with at least one answer
df_to_summary = df_to.groupby('Tool').count()['Question_title'].reset_index()
df_to_answered_summary = df_to_answered.groupby(
    'Tool').count()['Question_title'].reset_index()

df_to_summary.columns = ['Tool', '#Question']
df_to_answered_summary.columns = ['Tool', '#Answered']

df_summary = df_to_summary.merge(df_to_answered_summary, on='Tool')

df_summary.to_csv(os.path.join(path_to, 'summary.csv'), index=False)
df_summary

,Tool,#Question,#Answered
0,Amazon SageMaker,528,167
1,Azure Machine Learning,1435,343
2,DVC,315,300
3,Domino,13,4
4,Guild AI,115,108
5,MLFlow,280,143
6,Polyaxon,43,34
7,SigOpt,15,7
8,Vertex AI,297,32
9,Weights & Biases,583,92


In [30]:
# After having the population for each tool and discussion channel, we then find out the minimum number of necessary samples with the [calculator](https://www.calculator.net/sample-size-calculator.html).
df_summary = pd.read_csv(os.path.join(path_to, 'summary.csv'))
df_summary


,Tool,#Question,#Answered,#Sample Question,#Sample Answered
0,Amazon SageMaker,528,167,223,117
1,Azure Machine Learning,1435,343,304,182
2,DVC,315,300,174,169
3,Domino,13,4,13,4
4,Guild AI,115,108,89,85
5,MLFlow,280,143,163,105
6,Polyaxon,43,34,39,32
7,SigOpt,15,7,15,7
8,Vertex AI,297,32,168,30
9,Weights & Biases,583,92,232,75


In [31]:
# sample tool-specific posts accordingly
for index, row in df_summary.iterrows():
    df_to[df_to['Tool'] == row['Tool']].sample(n=row['#Sample Question'], random_state=0).to_json(
        os.path.join(path_to_sampled_questions, row['Tool'] + '.json'), orient='records', indent=4)
    df_to_answered[df_to_answered['Tool'] == row['Tool']].sample(n=row['#Sample Answered'], random_state=0).to_json(
        os.path.join(path_to_sampled_answered, row['Tool'] + '.json'), orient='records', indent=4)

In [6]:
df = pd.read_json(os.path.join(
    path_so_raw, 'bq-results-20221222-171427-1671729301167.json'), lines=True)
df.head()


,Question_id,Question_title,Question_body,Question_answer_count,Question_comment_count,Question_creation_time,Question_score,Question_tags,Question_view_count,Owner_creation_time,...,Owner_up_votes,Owner_down_votes,Owner_views,Question_last_edit_time,Answer_body,Answer_comment_count,Answer_creation_time,Answer_score,Question_favorite_count,Answer_last_edit_time
0,68573290,How to close Gremlin session client without ex...,"<p>I'm trying to use <a href=""https://docs.aws...",1,0,2021-07-29 09:17:40.850000+00:00,0,transactions|gremlin|tinkerpop3|amazon-neptune...,195,2013-06-04 08:43:04.927000+00:00,...,0,0,4,NaT,NaN,NaN,NaT,NaN,NaN,NaT
1,71711284,Py4JJavaError while loading Jar file in SageMa...,<p>I am having issue when I try to load jar fi...,0,1,2022-04-01 18:21:43.433000+00:00,0,scala|jupyter-notebook|jvm|amazon-sagemaker|py4j,184,2022-03-30 14:13:10.150000+00:00,...,0,0,2,2022-04-01 18:32:39.450000+00:00,NaN,NaN,NaT,NaN,NaN,NaT
2,42766263,How to Find Azure ML Experiment based on Deplo...,<p>I have a list of ML experiments which I hav...,1,0,2017-03-13 14:35:50.577000+00:00,7,web-services|azure|azure-machine-learning-studio,224,2013-08-28 15:24:36.330000+00:00,...,18,0,43,NaT,<p>The <em>new</em> web service does not store...,0.0,2017-03-23 15:38:46.690000+00:00,1.0,NaN,NaT
3,64137409,How can I create an Azure dataset in Azure ML ...,<p>I'm trying to load files as a dataset in th...,2,0,2020-09-30 12:18:12.143000+00:00,2,azure|apache-spark|parquet|azure-machine-learn...,178,2015-02-11 07:34:40.283000+00:00,...,19,2,125,NaT,<p>Thanks for the feedback. You can use globin...,1.0,2020-09-30 16:39:04.070000+00:00,2.0,NaN,NaT
4,1940103,How suitable is a DVCS for the corporate envir...,"<p>I've been using SVN for some time now, and ...",11,0,2009-12-21 13:27:42.327000+00:00,9,version-control|dvcs,836,2009-11-21 21:24:44.097000+00:00,...,4,2,229,2010-03-29 04:36:12.517000+00:00,<p>With DVCS's people can maintain their own l...,4.0,2009-12-21 13:37:05.467000+00:00,4.0,4.0,NaT


In [7]:
# create tag collection
tags = []
for key, value in tool2tag.items():
    tags.extend(value)

In [8]:
# split tags
df['Question_valid_tags'] = [[] for _ in range(len(df))]
for index, row in df.iterrows():
    row_tags = set(str(row['Question_tags']).strip().split('|'))
    df.at[index, 'Question_valid_tags'] = list(row_tags.intersection(tags))


In [9]:
# count post number with different tags
arity = 0
while True:
    post_number = df[df['Question_valid_tags'].map(len) > arity].shape[0]
    if post_number < 1:
        break
    arity = arity + 1
    print(f'Posts with at least {arity} tags has {post_number} in total.')

Posts with at least 1 tags has 4981 in total.
Posts with at least 2 tags has 213 in total.
Posts with at least 3 tags has 16 in total.


In [10]:
# exclude Stack Overflow posts with unrelated tags
df_valid = df[df['Question_valid_tags'].map(len) > 0]
df_valid['Question_link'] = df_valid['Question_id'].apply(
    lambda x: f'https://stackoverflow.com/questions/{x}')


C:\Users\zhimi\AppData\Local\Temp\ipykernel_11516\729265963.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid['Question_link'] = df_valid['Question_id'].apply(


In [16]:
# exclude Stack Overflow posts with negative upvote count
df_qualified = df_valid[df_valid['Question_score'] > -1]

In [13]:
# create a map from tag to tool
tag2tool = {}
for key, value in tool2tag.items():
    for elem in value:
        tag2tool.setdefault(elem, key)

In [ ]:
# extract Stack Overflow post collection with multiple tags based on the tool map
for index, row in df_qualified.iterrows():
    tags = set()
    for tag in row['Question_valid_tags']:
        tags.add(tag2tool[tag])
    df_qualified.at[index, 'Question_valid_tags'] = sorted(list(tags))


In [17]:
# check how the posts with more than one tags look like
df_multiply_tagged = df_qualified[df_qualified['Question_valid_tags'].map(
    len) > 1]
df_multiply_tagged['Question_valid_tags'].value_counts()


[Amazon SageMaker, MLFlow]          16
[Azure Machine Learning, MLFlow]    13
[DVC, MLFlow]                        1
[DVC, Pachyderm]                     1
Name: Question_valid_tags, dtype: int64

In [20]:
# create Stack Overflow post collection with exclusive tags
multiply_tagged_posts_split = []
df_qualified.assign(Question_exclusive_tag='')

for index, row in df_qualified.iterrows():
    tags = row['Question_valid_tags']
    df_qualified.at[index, 'Question_exclusive_tag'] = tags[0]
    if len(tags) > 1:
        for tag in tags[1:]:
            series = row.copy()
            series['Question_exclusive_tag'] = tag
            multiply_tagged_posts_split.append(series)

df_multiply_tagged_posts_split = pd.DataFrame(multiply_tagged_posts_split)
df_qualified_exclusive_tagged = pd.concat(
    [df_qualified, df_multiply_tagged_posts_split], ignore_index=True)
del df_qualified_exclusive_tagged['Question_valid_tags']
df_qualified_exclusive_tagged.to_json(os.path.join(
    path_so_filtered, 'non_negative_scored_exclusively_tagged.json'), indent=4, orient='records')


In [21]:
# keep Stack Overflow posts with accepted answers
df_qualified_exclusive_tagged_completed = df_qualified_exclusive_tagged.dropna(
    subset=['Answer_body'])
df_qualified_exclusive_tagged_completed.to_json(os.path.join(
    path_so_filtered, 'completed_non_negative_scored_exclusively_tagged.json'), indent=4, orient='records')

In [97]:
df_so_summary = df_qualified_exclusive_tagged.groupby(
    'Question_exclusive_tag').count()['Question_id'].reset_index()
df_so_answered_summary = df_qualified_exclusive_tagged_completed.groupby(
    'Question_exclusive_tag').count()['Question_id'].reset_index()

df_so_summary.columns = ['Tool', '#Question']
df_so_answered_summary.columns = ['Tool', '#Answered']

df_summary = pd.merge(df_so_summary, df_so_answered_summary, on='Tool')
df_summary.to_csv(os.path.join(path_so, 'summary.csv'), index=False)
df_summary

,Tool,#Question,#Answered
0,Amazon SageMaker,2233,737
1,Azure Machine Learning,1530,586
2,ClearML,40,20
3,Comet,10,4
4,DVC,91,49
5,MLFlow,551,129
6,Neptune,8,3
7,Pachyderm,7,2
8,Vertex AI,341,112
9,Weights & Biases,77,22


In [22]:
# After having the population for each tool and discussion channel, we then find out the minimum number of necessary samples with the [calculator](https://www.calculator.net/sample-size-calculator.html).
df_summary = pd.read_csv(os.path.join(path_so, 'summary.csv'))
df_summary


,Tool,#Question,#Answered,#Sample Question,#Sample Answered
0,Amazon SageMaker,2233,737,328,253
1,Azure Machine Learning,1530,586,308,233
2,ClearML,40,20,37,20
3,Comet,10,4,10,4
4,DVC,91,49,74,44
5,MLFlow,551,129,227,97
6,Neptune,8,3,8,3
7,Pachyderm,7,2,7,2
8,Vertex AI,341,112,181,87
9,Weights & Biases,77,22,65,21


In [23]:
# sample Stack Overflow posts accordingly
for index, row in df_summary.iterrows():
    df_qualified_exclusive_tagged[df_qualified_exclusive_tagged['Question_exclusive_tag'] == row['Tool']].sample(n=row['#Sample Question'], random_state=0).to_json(
        os.path.join(path_so_sampled_questions, row['Tool'] + '.json'), orient='records', indent=4)
    df_qualified_exclusive_tagged_completed[df_qualified_exclusive_tagged_completed['Question_exclusive_tag'] == row['Tool']].sample(n=row['#Sample Answered'], random_state=0).to_json(
        os.path.join(path_so_sampled_answered, row['Tool'] + '.json'), orient='records', indent=4)